# Module 2: Data wrangling using `pandas`

## Overview: Clean vs. dirty geochronology datasets
This notebook will take you through python code to import, sort, and make some very basic plots of some U-Pb detrital zircon geochronology data using `pandas`. We will look at one nicely cleaned dataset, and anothrer dataset that is messier, and needs some wrangling before we can play with it.

For questions on this notebook, ask them on the [GEOL 557 slack](https://join.slack.com/t/minesgeo/shared_invite/zt-cqawm4lu-Zcfpf4mBLwjnksY6_umlKA)<a href="https://join.slack.com/t/minesgeo/shared_invite/zt-cqawm4lu-Zcfpf4mBLwjnksY6_umlKA">
<img src="https://cdn.brandfolder.io/5H442O3W/as/pl546j-7le8zk-ex8w65/Slack_RGB.svg" alt="Go to the GEOl 557 slack" width="100">
</a>

## Instructions
Work through this notebook - there will be several places where you need to fill-in-the-blank or write some code into an open cell. When you are finished, make sure to use the Colab menu (not the browser-level menu) to do the following:
- Expand all the sections - in the Colab menu, choose View --> Expand sections) 
- Save the notebook as a pdf, again using the Colab menu, using File --> Print --> Save as PDF. 

--- 
## Course
**GEOL 557 Earth Resource Data Science I: Fundamentals**. GEOL 557 forms part 2 of the four-part course series for the "Earth Resource Data Science" online graduate certificate at Mines - [learn more about the certificate here](https://online.mines.edu/er/)

Notebook created by **Zane Jobe** and **Thomas Martin**, [CoRE research group](https://core.mines.edu), Colorado School of Mines

[![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/ZaneJobe.svg?style=social&label=Follow%20%40ZaneJobe)](https://twitter.com/ZaneJobe)
and [![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/ThomasM_geo.svg?style=social&label=Follow%20%40ThomasM_geo)](https://twitter.com/ThomasM_geo) on Twitter 

# TO DO: change path to GEOL_557

In [ ]:
import pandas as pd # this imports pandas to this notebook
import numpy as np
from matplotlib import pyplot as plt

# these next two things shuoldnt need to be changed if you set up your Google Drive folder correctly (see Module 1)
folder_path = '../1_data/' # makes a path
file_name = 'Sharman_ExampleDataset_1.xlsx' # file name

## Let's load in a dataset
The data we want is located here https://github.com/grsharman/detritalPy/blob/master/detritalPy/example-data/ExampleDataset_1.xlsx

It is downloaded and saved in the Google Drive folder. 

We load it with the pandas `read_xls` function into a DataFrame called `df`:

In [ ]:
df=pd.read_excel(folder_path + file_name, sheet_name='ZrUPb') # uses pandas to read in the csv as a 'DataFrame' called df
df.head()

Nice! Let's make sure the dtypes are correct:

In [ ]:
df.info()

Excellent - I wish all data files were formatted like that. Let's check out a description of the data:

In [ ]:
df.describe()

In [ ]:
df.groupby('Sample_ID').BestAge.describe()

In [ ]:
df.groupby('Sample_ID').size() # number of rows per sample

In [ ]:
df.groupby('Sample_ID').BestAge.max() # oldest Age per sample (could also do min, mean, median, etc.)

In [ ]:
df.BestAge.hist() # all ages as a histogram

In [ ]:
# each sample as a separate box plot
df.groupby('Sample_ID').boxplot(column=['BestAge'], grid=False, showfliers=False, figsize=[10,20], sharey=True)
plt.show()

In [ ]:
df.plot(x='BestAge', y='BestAge_err', style='.');

### Now you try
Make a plot using one of the built-in pandas plotting methods - anything you want! 

In [ ]:
# your code goes here

![I did it](https://media1.tenor.com/images/a5747f7b1d84287ca4a62e8a428d51ae/tenor.gif?itemid=4990241)

# Some not-so-easy geochronology data

Ok, now it's time for the not-so-easy dataset. Here goes:

In [ ]:
# these next two things shuoldnt need to be changed if you set up your Google Drive folder correctly (see Module 1)
folder_path = '../1_data/' # makes a path
file_name = 'Daniels_GSA_2017304_appendix2.xlsx' # file name

## Let's load in a dataset
The data we want is located here https://doi.org/10.1130/B31757.1 It is the Appendix 2 file ("Supplemental Information 2" down at the bottom   of the   page). Two tabs in that xlsx are useful to us, the "high throughput" and "low throughput" tabs, which are U-Pb ages from detrital zircons from the Magallanes Basin, Chile. Also, by default, `pandas` imports the first sheet, but we also want to specify that, so we use `sheet_name` for that. 

Before we load the data, the Excel file is weird, and has a linked IsoPlot function in the file that needs to be broken/deleted. I took care of breaking this link and saving as a new file, which fixes weird encryption issues with Excel format. Yet another reason to just use a csv file... 

We load it with the pandas `read_xls` function into a DataFrame called `df`:

In [ ]:
df=pd.read_excel(folder_path + file_name, sheet_name='HighThroughputAges - LA-ICP-MS') # uses pandas to read in the csv as a 'DataFrame' called df
df.head()

Hmm, that doesnt look right. There are a few header lines, which are messing up the key names (currently in row `1`. You could go into Excel and delete them, but it's good practice to modify the input file as little as possible. Also, there are 13 samples and two tabs, so that means you would have to fix these issues 26 times manually, which is sub-optimal. Let's automate the boring stuff!

In [ ]:
df=pd.read_excel(folder_path + file_name, sheet_name='HighThroughputAges - LA-ICP-MS', skiprows=2) # uses pandas to read in the csv as a 'DataFrame' called df
df.head()

OK, that's a bit better, but still not great. If you look at the Excel, it has empty rows to separate the samples, and merged rows to indicate column names and subnames. This looks snazzy in Excel, but is a pain when doing data analysis because it means that one row is empty (hence the `Unnamed:1` as the second column name. 

First, let's get rid of the empty rows:

In [ ]:
print('df has length', len(df), 'before dropping rows with no data')
df.dropna(how='all', inplace=True)
print('df has length', len(df), 'after dropping rows with no data')

Now to deal with the column names. Ideally, we would rename the column names so that the name and the subname is in each, like this first example:

In [ ]:
df.rename({'Data for Tera-Wasserburg plot2'	: 'TeraWasserburg_238U/206Pb'},axis=1, inplace=True)
df.head()

There are smarter, more automated ways to do that (using a loop or `.apply` methods), but that's a topic for another day. 

In this case, we don't really want to fool around with all those columns anyways - what we are really interested in are the ages, and so we can just drop all the columns that don't have age information. 

In [ ]:
#renames the columns first
col_dict = {'15-CC-01 HT':'Sample', 
            'Unnamed: 1':'Spot', 
            'Dates':'Age_207Pb/206Pb', 
            'Unnamed: 25':'Age_206Pb/238U', 
            'Unnamed: 28':'Age_207Pb/235Pb', 
            'Accepted Dates4':'Age_Accepted'
           }

df.rename(col_dict, axis=1, inplace=True) # axis 1 acts on the columns

# now let's drop all the columns we didnt rename
col_list = list(col_dict.values())
df = df.filter(col_list, axis=1) # reassign it
df.head()

In [ ]:
# get rid of first line
df.drop(0,axis=0, inplace=True) # axis 0 works on the rows
df.head()

And lastly, from the way that xls was formatted, there should be some extra rows that only had values in the first column that we renamed to `Sample`, so let's look at that column:

In [ ]:
print(df.Sample.value_counts())
print(df.Sample.value_counts().values)


Whoa, lots of weird things there. The way that xls is formatted is really not great! It would be smart to replace the sample names beginning with `UK` with the ones beginning with `15-`, but for now we won't mess with that. Let's just get rid of the rows that only have data in the `Sample` column, and no other data:

In [ ]:
# Which columns have NaNs
df.isnull().sum()

See the 32 in Spot? There shouldn't be any NaNs there either, if each `Spot` is part of a `Sample`. So, let's get rid of rows where Spot is NaN

In [ ]:
# get rid of all rows where Spot is NaN
df.dropna(axis=0, subset = ['Spot'], inplace=True)

# now check the samples
print(df.isnull().sum())
print(df.Sample.value_counts())

Great! That means we have 12 Samples, , with around 640 ages per sample. 

Last thing we need to do is to convert the `objects` to other dtypes:

In [ ]:
df.info()

In [ ]:
# get rid of strings in the age columns
df[df.columns[2:6]] = df[df.columns[2:6]].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float).dropna()
# you can read about the coerce method on the pandas website

In [ ]:
df.info()

### Whew! 
Ok, that took a bit of work, and you might think you can do this faster in Excel, and you are probably correct. but think about if instead of one spreadsheet, you had 55 or 550 of these spreadsheets to deal with - still want to do it in Excel? 

### Now let's try a simple plot:

In [ ]:
df.plot.scatter(x='Age_207Pb/206Pb', y='Age_206Pb/238U')

In [ ]:
df['Age_206Pb/238U'].max()

In [ ]:
df.plot.scatter(x='Age_207Pb/206Pb', y='Age_206Pb/238U', xlim=[0, 4000], ylim=[0,4000])

In [ ]:
### Now you try - for example, do a `describe()` on a column and a `hist` of ages (you can choose the column you want to plot)

In [ ]:
# your code here

### Now let's export that clean dataset as a csv
Search the pandas documention to figure out how to export a csv of this dataset you have just cleaned up

In [ ]:
# your code here

![Robert Redford](https://media1.tenor.com/images/3952a85da6e63c7755607a40a4bc975f/tenor.gif?itemid=4959267)